### Entrenar un DDSP autoencoder.

Este notebook permite entrenar un autoencoder para posteriormente realizar transferencia de timbre/tono con [DDSP Timbre Transfer Colab](https://colab.research.google.com/github/magenta/ddsp/blob/master/ddsp/colab/demos/timbre_transfer.ipynb).

traducido por @hypereikon


In [ ]:
#@markdown instalar dependencias
%tensorflow_version 2.x
!pip install -qU ddsp[data_preparation]==1.0.1
# Initialize global path for using google drive. 
DRIVE_DIR = ''

In [ ]:
#@markdown Montar google drive
from google.colab import drive
drive.mount('/content/drive')

#### Setear la carpeta base
En drive, sube todos los archivos de audio mono (.wav, .mp3) que quieras entrenar a una carpeta.
Funciona bien con ~20 min de sonido *monofonico*, es decir un instrumento.

En el explorador de archivos de la izquierda ubica la carpeta con los archivos y dale a "*copiar ruta*", luego pegala abajo y ejecuta la celda.

In [ ]:
#@markdown carpeta donde estan los archivos de audio
DRIVE_DIR = '/content/drive/MyDrive/ddsp anticueca' #@param {type: "string"}

import os
assert os.path.exists(DRIVE_DIR)
print('Drive Folder Exists:', DRIVE_DIR)


In [ ]:
#@markdown Crear directorios para guardar el modelo y los datos
AUDIO_DIR = 'data/audio'
AUDIO_FILEPATTERN = AUDIO_DIR + '/*'
!mkdir -p $AUDIO_DIR

if DRIVE_DIR:
  SAVE_DIR = os.path.join(DRIVE_DIR, 'ddsp-solo-instrument')
else:
  SAVE_DIR = '/content/models/ddsp-solo-instrument'
!mkdir -p "$SAVE_DIR"

#### Preparar el dataset


#### Cargar audio para entrenar
###si continuas entrenando ignora esta celda

Ubicar archivos de audio para entrenar el modelo. 

In [ ]:
#@markdown Cargar audio / Si continuas entrenando ignora esta celda
import glob
import os
from ddsp.colab import colab_utils

if DRIVE_DIR:
  mp3_files = glob.glob(os.path.join(DRIVE_DIR, '*.mp3'))
  wav_files = glob.glob(os.path.join(DRIVE_DIR, '*.wav'))
  audio_files = mp3_files + wav_files
else:
  audio_files, _ = colab_utils.upload()

for fname in audio_files:
  target_name = os.path.join(AUDIO_DIR, 
                             os.path.basename(fname).replace(' ', '_'))
  print('Copying {} to {}'.format(fname, target_name))
  !cp "$fname" $target_name

### Preprocesar audio a dataset TFRecord
Necesitamos procesar el audio a un formato adecuado para entrenar.
Esto implica inferir la frecuencia fundamental y loudness. Estos parametros se guardan en un archivo TFRecord.

Si continuas entrenando ubicara los TFRecords en tu drive automaticamente.

In [ ]:
#@markdown crear/ubicar TFRecord
import glob
import os

TRAIN_TFRECORD = 'data/train.tfrecord'
TRAIN_TFRECORD_FILEPATTERN = TRAIN_TFRECORD + '*'

# Copy dataset from drive if dataset has already been created.
drive_data_dir = os.path.join(DRIVE_DIR, 'data') 
drive_dataset_files = glob.glob(drive_data_dir + '/*')

if DRIVE_DIR and len(drive_dataset_files) > 0:
  !cp "$drive_data_dir"/* data/

else:
  # Make a new dataset.
  if not glob.glob(AUDIO_FILEPATTERN):
    raise ValueError('No audio files found. Please use the previous cell to '
                    'upload.')

  !ddsp_prepare_tfrecord \
    --input_audio_filepatterns=$AUDIO_FILEPATTERN \
    --output_tfrecord_path=$TRAIN_TFRECORD \
    --num_shards=10 \
    --alsologtostderr

  # Copy dataset to drive for safe-keeping.
  if DRIVE_DIR:
    !mkdir "$drive_data_dir"/
    print('Saving to {}'.format(drive_data_dir))
    !cp $TRAIN_TFRECORD_FILEPATTERN "$drive_data_dir"/

In [ ]:
#@markdown guardar un .pkl con las estadisticas del dataset para transferencia de timbre.
from ddsp.colab import colab_utils
import ddsp.training

data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_dataset(shuffle=False)
PICKLE_FILE_PATH = os.path.join(SAVE_DIR, 'dataset_statistics.pkl')

colab_utils.save_dataset_statistics(data_provider, PICKLE_FILE_PATH, batch_size=1)

In [ ]:
#@markdown cargar el dataset a la libreria ddsp y ver un ejemplo
from ddsp.colab import colab_utils
import ddsp.training
from matplotlib import pyplot as plt
import numpy as np

data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_dataset(shuffle=False)

try:
  ex = next(iter(dataset))
except StopIteration:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

colab_utils.specplot(ex['audio'])
colab_utils.play(ex['audio'])

f, ax = plt.subplots(3, 1, figsize=(14, 4))
x = np.linspace(0, 4.0, 1000)
ax[0].set_ylabel('loudness_db')
ax[0].plot(x, ex['loudness_db'])
ax[1].set_ylabel('F0_Hz')
ax[1].set_xlabel('seconds')
ax[1].plot(x, ex['f0_hz'])
ax[2].set_ylabel('F0_confidence')
ax[2].set_xlabel('seconds')
ax[2].plot(x, ex['f0_confidence'])


## Entrenar modelo

In [ ]:
#@markdown Primero, activar Tensorboard para monitorear las perdidas y el desarrollo del entrenamiento.
%reload_ext tensorboard
import tensorboard as tb
tb.notebook.start('--logdir "{}"'.format(SAVE_DIR))

### Notas para el entrenamiento:
* Los modelos performan bien cuando la perdida (loss) llega a ~4.5-5.9
* Dependiendo del dataset esto puede demorar entre 5k-30k pasos (steps).
* Por defecto esta seteado a 30k, pero puedes parar en cualquier momento, y para la tranferencia de timbre, es mejor parar antes de que la perdida baje mucho de ~5.0 para evitar *overfitting*.
* En colab, esto puede demorar entre 3-20 horas.
* Por defecto, los checkpoints se guardaran cada 300 pasos con un maximo de 10 checkpoints (~60MB/checkpoint). Ajusta estos parametros como te convenga.

In [ ]:
#@markdown entrenar
!ddsp_run \
  --mode=train \
  --alsologtostderr \
  --save_dir="$SAVE_DIR" \
  --gin_file=models/solo_instrument.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="TFRecordProvider.file_pattern='$TRAIN_TFRECORD_FILEPATTERN'" \
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=30000" \
  --gin_param="train_util.train.steps_per_save=500" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=5"

## Resintesis
Revisa que tan bien reconstruye los datos de entrenamiento.

In [ ]:
#@markdown resintesis
from ddsp.colab.colab_utils import play, specplot
import ddsp.training
import gin
from matplotlib import pyplot as plt
import numpy as np

data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_batch(batch_size=1, shuffle=False)

try:
  batch = next(iter(dataset))
except OutOfRangeError:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

# Parse the gin config.
gin_file = os.path.join(SAVE_DIR, 'operative_config-0.gin')
gin.parse_config_file(gin_file)

# Load model
model = ddsp.training.models.Autoencoder()
model.restore(SAVE_DIR)

# Resynthesize audio.
outputs = model(batch, training=False)
audio_gen = model.get_audio_from_outputs(outputs)
audio = batch['audio']

print('Original Audio')
specplot(audio)
play(audio)

print('Resynthesis')
specplot(audio_gen)
play(audio_gen)

## Descargar checkpoint

Con la siguiente celda puedes descargar el checkpoint final para usarlo con [DDSP Timbre Tranfer Colab](https://colab.research.google.com/github/magenta/ddsp/blob/master/ddsp/colab/demos/timbre_transfer.ipynb).

In [ ]:
#@markdown descargar
from ddsp.colab import colab_utils
import tensorflow as tf
import os

CHECKPOINT_ZIP = 'my_solo_instrument.zip'
latest_checkpoint_fname = os.path.basename(tf.train.latest_checkpoint(SAVE_DIR))
!cd "$SAVE_DIR" && zip $CHECKPOINT_ZIP $latest_checkpoint_fname* operative_config-0.gin dataset_statistics.pkl
!cp "$SAVE_DIR/$CHECKPOINT_ZIP" ./
colab_utils.download(CHECKPOINT_ZIP)